In [1]:
import subprocess
import cv2
import matplotlib.pyplot as plt
import subprocess
import time
import os
import glob

In [2]:
def get_image_paths(image_folder):
    # Get the list of all image files in the specified folder
    image_paths = glob.glob(os.path.join(image_folder, '*'))
    return image_paths

In [3]:
def run_ilastik(input_path, ilastik_script_path, project_path, result_base_path, export_source="Simple Segmentation", output_format="png"):
    """
    This function runs the Ilastik software in headless mode with the specified parameters.

    Parameters:
    input_path (str): The path to the image file or folder to be processed.
    ilastik_script_path (str): The path to the Ilastik script.
    project_path (str): The path to the Ilastik project file.
    result_base_path (str): The base path where the result will be saved.
    export_source (str, optional): The type of data to export. Default is "Simple Segmentation". Must be one of ["Probabilities", "Simple Segmentation", "Uncertainty", "Features", "Labels"].
    output_format (str, optional): The format of the output file. Default is "png". Must be one of ["bmp", "gif", "hdr", "jpeg", "jpg", "pbm", "pgm", "png", "pnm", "ppm", "ras", "tif", "tiff", "xv", "bmp sequence", "gif sequence", "hdr sequence", "jpeg sequence", "jpg sequence", "pbm sequence", "pgm sequence", "png sequence", "pnm sequence", "ppm sequence", "ras sequence", "tif sequence", "tiff sequence", "xv sequence", "multipage tiff", "multipage tiff sequence", "hdf5", "compressed hdf5", "numpy, dvid"].

    Returns:
    None

    Raises:
    ValueError: If export_source or output_format is not in the allowed list.
    subprocess.CalledProcessError: If there is an error during the execution of the Ilastik command.
    """
    
    ALLOWED_SOURCES = ["Probabilities", "Simple Segmentation", "Uncertainty", "Features", "Labels"]
    ALLOWED_FORMATS = ["bmp", "gif", "hdr", "jpeg", "jpg", "pbm", "pgm", "png", "pnm", "ppm", "ras",
                    "tif", "tiff", "xv", "bmp sequence", "gif sequence", "hdr sequence", "jpeg sequence",
                    "jpg sequence", "pbm sequence", "pgm sequence", "png sequence", "pnm sequence",
                    "ppm sequence", "ras sequence", "tif sequence", "tiff sequence", "xv sequence",
                    "multipage tiff", "multipage tiff sequence", "hdf5", "compressed hdf5", "numpy, dvid"]
    
    if export_source not in ALLOWED_SOURCES:
        raise ValueError(f"Invalid export_source. Allowed values are {ALLOWED_SOURCES}")
    
    if output_format not in ALLOWED_FORMATS:
        raise ValueError(f"Invalid output_format. Allowed values are {ALLOWED_FORMATS}")
    
    # Check if result_base_path exists, if not, create it
    if not os.path.exists(result_base_path):
        os.makedirs(result_base_path)

# Check if input_path is a directory or a file
    if os.path.isdir(input_path):
        image_arg = get_image_paths(input_path)
        # Arguments to execute Ilastik in headless mode
        ilastik_args = [
            ilastik_script_path,
            "--headless",
            "--project=" + project_path,
            "--export_source=" + export_source,
            "--output_format=" + output_format,
            "--output_filename_format=" + result_base_path + "{nickname}_" + export_source.replace(" ", "_"),
            *image_arg
        ]
    else:
        image_arg = input_path
        # Arguments to execute Ilastik in headless mode
        ilastik_args = [
            ilastik_script_path,
            "--headless",
            "--project=" + project_path,
            "--export_source=" + export_source,
            "--output_format=" + output_format,
            "--output_filename_format=" + result_base_path + "{nickname}_" + export_source.replace(" ", "_"),
            image_arg
        ]

    # Execute the Ilastik command in headless mode with the specified arguments
    try:
        subprocess.run(ilastik_args, check=True)
        print(f"Conversion of {input_path} completed successfully")
    except subprocess.CalledProcessError as e:
        print("Error during conversion : ", e)
        raise RuntimeError("Error during Ilastik execution. See console output for details.")

In [5]:
def display_images(image_path, result_base_path, export_source, output_format):
    """
    This function reads an original image and its corresponding result image, concatenates them horizontally, and displays the concatenated image.

    Parameters:
    image_path (str): The path to the original image file.
    result_base_path (str): The base path where the result image is saved.
    export_source (str): The type of data that was exported to create the result image.
    output_format (str): The format of the result image file.

    Returns:
    None

    Note:
    The function assumes that the result image file is named by replacing the ".jpg" extension of the original image file name with "_{export_source}.{output_format}", where {export_source} is the export_source parameter with spaces replaced by underscores.

    The function uses OpenCV to read the images and Matplotlib to display the concatenated image. The images are concatenated horizontally using cv2.hconcat. The concatenated image is displayed in RGB color space, so the function converts the color space from BGR to RGB using cv2.cvtColor.
    """
    
    input_filename = os.path.basename(image_path)

    result_path = os.path.join(result_base_path, input_filename.replace(".jpg", "_" + export_source.replace(" ", "_") + "." + output_format))

    img = cv2.imread(image_path)
    result = cv2.imread(result_path)

    concatenated_img = cv2.hconcat([img, result])

    plt.figure(figsize=(15, 15))
    plt.imshow(cv2.cvtColor(concatenated_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [40]:
input_path = "model_evaluation/test_images"
ilastik_script_path = "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/run_ilastik.sh"
project_path = "labelisation3.ilp"
result_base_path = "model_evaluation/model_predictions/"
export_source = "Simple Segmentation"
output_format = "png"

start_time = time.time()
run_ilastik(input_path = input_path,
            ilastik_script_path = ilastik_script_path,
            project_path = project_path,
            result_base_path = result_base_path,
            export_source = export_source,
            output_format = output_format)
end_time = time.time()
print(f"Execution time: {(end_time - start_time):.3f} seconds")

'''
display_images(image_path = image_path,
               result_base_path = result_base_path,
               export_source = export_source,
               output_format = output_format)
'''

INFO ilastik.app: Using tiktorch executable: ['/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/bin/python', '-m', 'tiktorch.server']
INFO ilastik.app: config file location: <none>
INFO ilastik.app: Starting ilastik from "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".
Starting ilastik from "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".


WARNING 2024-04-10 15:29:38,144 opConservationTracking 56124 8670007872 Could not find any ILP solver
WARNING 2024-04-10 15:29:38,154 opStructuredTracking 56124 8670007872 Could not find any ILP solver
WARNING 2024-04-10 15:29:38,157 structuredTrackingWorkflow 56124 8670007872 Could not find any learning solver. Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!


INFO ilastik.shell.projectManager: Opening Project: /Users/titouanlegourrierec/Desktop/labelisation3.ilp
INFO ilastik.workflows.pixelClassification.pixelClassificationWorkflow: Beginning Batch Processing
INFO ilastik.applets.dataSelection.dataSelectionApplet: Using axistags from previous lane: [y x c, None]
INFO ilastik.applets.batchProcessing.batchProcessingApplet: Exporting to /Users/titouanlegourrierec/Desktop/ilastik_result/664_leaf3_Simple_Segmentation.png
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per pixel is 212.0B * safety factor (2.0)
INFO lazyflow.utility.bigRequestStreamer: determining blockshape assuming available_ram is 10.9GiB, split between 8 threads
INFO lazyflow.utility.bigRequestStreamer: Chose blockshape: (4169, 831, 4)
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per block is 1.4GiB
DEBUG lazyflow.operators.classifierOperators: Features took 1.6230820000000001 seconds. Prediction took 10.302567 seconds. Subregion: start '[0, 0, 0

'\ndisplay_images(image_path = image_path,\n               result_base_path = result_base_path,\n               export_source = export_source,\n               output_format = output_format)\n'

In [17]:
input_path = "/Users/titouanlegourrierec/Desktop/new_ANNOT/YUV"
ilastik_script_path = "/Applications/ilastik-1.4.1b15-arm64-OSX.app/Contents/ilastik-release/run_ilastik.sh"
project_path = "/Users/titouanlegourrierec/Desktop/color_space_evaluation_OK 2/YUV/new_YUV.ilp"
result_base_path = "/Users/titouanlegourrierec/Desktop/new_ANNOT/new_result_YUV/"
export_source = "Simple Segmentation"
output_format = "png"

start_time = time.time()
run_ilastik(input_path = input_path,
            ilastik_script_path = ilastik_script_path,
            project_path = project_path,
            result_base_path = result_base_path,
            export_source = export_source,
            output_format = output_format)
end_time = time.time()
print(f"Execution time: {(end_time - start_time):.3f} seconds")

INFO ilastik.app: Using tiktorch executable: ['/Applications/ilastik-1.4.1b15-arm64-OSX.app/Contents/ilastik-release/bin/python', '-m', 'tiktorch.server']
INFO ilastik.app: config file location: <none>
INFO ilastik.app: Starting ilastik from "/Applications/ilastik-1.4.1b15-arm64-OSX.app/Contents/ilastik-release/lib/python3.9".
Starting ilastik from "/Applications/ilastik-1.4.1b15-arm64-OSX.app/Contents/ilastik-release/lib/python3.9".


WARNING 2024-04-25 23:12:14,231 opConservationTracking 85073 8211348160 Could not find any ILP solver
WARNING 2024-04-25 23:12:14,239 opStructuredTracking 85073 8211348160 Could not find any ILP solver
WARNING 2024-04-25 23:12:14,241 structuredTrackingWorkflow 85073 8211348160 Could not find any learning solver. Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!


INFO ilastik.shell.projectManager: Opening Project: /Users/titouanlegourrierec/Desktop/color_space_evaluation_OK 2/YUV/new_YUV.ilp
INFO ilastik.workflows.pixelClassification.pixelClassificationWorkflow: Beginning Batch Processing
INFO ilastik.applets.dataSelection.dataSelectionApplet: Using axistags from previous lane: [y x c, None]
INFO ilastik.applets.batchProcessing.batchProcessingApplet: Exporting to /Users/titouanlegourrierec/Desktop/new_ANNOT/new_result_YUV/260_leaf4_Simple_Segmentation.png
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per pixel is 248.0B * safety factor (2.0)
INFO lazyflow.utility.bigRequestStreamer: determining blockshape assuming available_ram is 9.8GiB, split between 8 threads
INFO lazyflow.utility.bigRequestStreamer: Chose blockshape: (1900, 1396, 4)
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per block is 1.2GiB
DEBUG lazyflow.operators.classifierOperators: Features took 0.555673 seconds. Prediction took 5.940189 seconds. S

In [5]:
from EasIlastik.run_ilastik import run_ilastik_probabilities

In [6]:
# define the path to the input folder containing the images to be segmented
input_path = "/Users/titouanlegourrierec/Desktop/new_ANNOT/images_RGB"

# define the path to the ilastik project file you want to use
project_path = "/Users/titouanlegourrierec/Desktop/color_space_evaluation_OK 2/RGB/BEST_OIDIUM.ilp"
#project_path = "/Users/titouanlegourrierec/Desktop/BEST_OIDIUM.ilp"

# define the path to the folder where the results should be saved
result_base_path = "/Users/titouanlegourrierec/Desktop/new_ANNOT/new_result_RGB/"

# define the threshold for the probabilities
threshold = 0

# define the color for the pixels below the threshold
below_threshold_color = [127, 127, 127]

# define the colors for the different channels
channel_colors = [[63, 63, 63], [127, 127, 127], [191, 191, 191], [255, 255, 255]]

run_ilastik_probabilities(input_path = input_path,
                          project_path = project_path,
                          result_base_path = result_base_path,
                          threshold = threshold,
                          below_threshold_color = below_threshold_color,
                          channel_colors = channel_colors)

INFO ilastik.app: Using tiktorch executable: ['/Applications/ilastik-1.4.1b15-arm64-OSX.app/Contents/ilastik-release/bin/python', '-m', 'tiktorch.server']
INFO ilastik.app: config file location: <none>
INFO ilastik.app: Starting ilastik from "/Applications/ilastik-1.4.1b15-arm64-OSX.app/Contents/ilastik-release/lib/python3.9".
Starting ilastik from "/Applications/ilastik-1.4.1b15-arm64-OSX.app/Contents/ilastik-release/lib/python3.9".


WARNING 2024-04-25 17:02:52,229 opConservationTracking 79950 8211348160 Could not find any ILP solver
WARNING 2024-04-25 17:02:52,238 opStructuredTracking 79950 8211348160 Could not find any ILP solver
WARNING 2024-04-25 17:02:52,241 structuredTrackingWorkflow 79950 8211348160 Could not find any learning solver. Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!


INFO ilastik.shell.projectManager: Opening Project: /Users/titouanlegourrierec/Desktop/color_space_evaluation_OK 2/RGB/BEST_OIDIUM.ilp
INFO ilastik.workflows.pixelClassification.pixelClassificationWorkflow: Beginning Batch Processing
INFO ilastik.applets.dataSelection.dataSelectionApplet: Using axistags from previous lane: [y x c, None]
INFO ilastik.applets.batchProcessing.batchProcessingApplet: Exporting to /Users/titouanlegourrierec/Desktop/new_ANNOT/new_result_RGB/260_leaf4_Probabilities.h5/exported_data
INFO lazyflow.operators.ioOperators.ioOperators.OpH5N5WriterBigDataset: Data shape: (8461, 1396, 4)
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per pixel is 224.0B * safety factor (2.0)
INFO lazyflow.utility.bigRequestStreamer: determining blockshape assuming available_ram is 9.8GiB, split between 8 threads
INFO lazyflow.utility.bigRequestStreamer: Chose blockshape: (2101, 1396, 4)
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per block is 1.2GiB
DE